As part of Kaggle competition 'Outbrain Click Prediction', this analysis was attempted. It turned out to be unsuccessful as I was not able to get the predictive model up and running in a PySpark cluster. Also, if I run this code any more, all my free credits in the Google Cloud Cluster will be used up. That said, I learnt how to start a Google cluster and run an IPython  notebook in it. I also learnt how to install other packages on a cluster straight from an IPython notebook. I obviously don't take credit for any of it but I guess it will be useful for me and for others when they want to use it. I know they are trivial things but the geek in me is happy.

## Installing Packages

Here, as shown below, a pip install pandas won't work. Rather use the statement os.system("sudo apt-get install python-pandas -y") and you'll not be facing a rough time like me figuring out how to install packages like numpy, pandas and so on. Also, I noticed that for some reason I had to append python- in front of the package name. It could be a way to tell the cluster that a Python package is being installed.

In [204]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
import sys

sys.path.append('/usr/lib/python2.7/dist-packages')

os.system("sudo apt-get install python-pandas -y")
os.system("sudo apt-get install python-numpy -y")
os.system("sudo apt-get install python-scipy -y")
os.system("sudo apt-get install python-sklearn -y")


import pandas
import numpy
import scipy
import sklearn
import pyspark.mllib as ml

In [210]:
from sklearn.ensemble import GradientBoostingClassifier

In [214]:
from pyspark.ml.classification import RandomForestClassifier

In [222]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier as RF
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## Data load

The following code first creates the necessary structures for the Spark data frames and then code is written to load the files stored in a Google File Store folder. Here, dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us represents the name of the File Store which contains all the input files. The code for loading data seems pretty self-explanatory.

In [114]:
page_views_schema = StructType(
                    [StructField("uuid", StringType(), True),
                    StructField("document_id", IntegerType(), True),
                    StructField("timestamp", IntegerType(), True),
                    StructField("platform", IntegerType(), True),
                    StructField("geo_location", StringType(), True),
                    StructField("traffic_source", IntegerType(), True)]
                    )

In [115]:
page_views_df = spark.read.schema(page_views_schema).options(header='true', inferschema='false', nullValue='\\N')
page_views_df = page_views_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/page_views.csv")

In [116]:
page_views_df.head()

Row(uuid=u'1fd5f051fba643', document_id=120, timestamp=31905835, platform=1, geo_location=u'RS', traffic_source=2)

In [117]:
document_categories_schema = StructType(
                    [StructField("document_id", IntegerType(), True),
                    StructField("category_id", IntegerType(), True),
                    StructField("confidence_level_cat", DoubleType(), True)]
                    )

In [118]:
document_categories_df = spark.read.schema(document_categories_schema).options(header='true', inferschema='false', nullValue='\\N')
document_categories_df = document_categories_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/documents_categories.csv")                

In [119]:
document_categories_df.head()

Row(document_id=1595802, category_id=1611, confidence_level_cat=0.92)

In [120]:
document_meta_schema = StructType(
                    [StructField("document_id", IntegerType(), True),
                    StructField("source_id", IntegerType(), True),
                    StructField("publisher_id", IntegerType(), True),
                    StructField("publish_time", TimestampType(), True)
                    ])

In [121]:
document_meta_df = spark.read.schema(document_meta_schema).options(header='true', inferschema='false', nullValue='\\N')
document_meta_df = document_meta_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/documents_meta.csv")                

In [122]:
document_meta_df.head()

Row(document_id=1595802, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 5, 0, 0))

In [123]:
clicks_train_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", IntegerType(), True),
                    StructField("clicked", IntegerType(), True)]
                    )

In [124]:
clicks_train_df = spark.read.schema(clicks_train_schema).options(header='true', inferschema='false', nullValue='\\N')
clicks_train_df = clicks_train_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/clicks_train.csv") 

In [248]:
clicks_train_df.head(15)

[Row(display_id=1, ad_id=42337, clicked=0),
 Row(display_id=1, ad_id=139684, clicked=0),
 Row(display_id=1, ad_id=144739, clicked=1),
 Row(display_id=1, ad_id=156824, clicked=0),
 Row(display_id=1, ad_id=279295, clicked=0),
 Row(display_id=1, ad_id=296965, clicked=0),
 Row(display_id=2, ad_id=125211, clicked=0),
 Row(display_id=2, ad_id=156535, clicked=0),
 Row(display_id=2, ad_id=169564, clicked=0),
 Row(display_id=2, ad_id=308455, clicked=1),
 Row(display_id=3, ad_id=71547, clicked=0),
 Row(display_id=3, ad_id=95814, clicked=0),
 Row(display_id=3, ad_id=152141, clicked=0),
 Row(display_id=3, ad_id=183846, clicked=0),
 Row(display_id=3, ad_id=228657, clicked=1)]

In [126]:
clicks_test_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", IntegerType(), True)]
                    )

In [127]:
clicks_test_df = spark.read.schema(clicks_test_schema).options(header='true', inferschema='false', nullValue='\\N')
clicks_test_df = clicks_test_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/clicks_test.csv") 

In [128]:
clicks_test_df.head(15)

[Row(display_id=16874594, ad_id=66758),
 Row(display_id=16874594, ad_id=150083),
 Row(display_id=16874594, ad_id=162754),
 Row(display_id=16874594, ad_id=170392),
 Row(display_id=16874594, ad_id=172888),
 Row(display_id=16874594, ad_id=180797),
 Row(display_id=16874595, ad_id=8846),
 Row(display_id=16874595, ad_id=30609),
 Row(display_id=16874595, ad_id=143982),
 Row(display_id=16874596, ad_id=11430),
 Row(display_id=16874596, ad_id=57197),
 Row(display_id=16874596, ad_id=132820),
 Row(display_id=16874596, ad_id=153260),
 Row(display_id=16874596, ad_id=173005),
 Row(display_id=16874596, ad_id=288385)]

In [195]:
events_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("uuid", StringType(), True),
                    StructField("document_id_e", IntegerType(), True),
                    StructField("timestamp", IntegerType(), True),
                    StructField("platform", IntegerType(), True),
                    StructField("geo_location", StringType(), True)]
                    )

In [196]:
events_df = spark.read.schema(events_schema).options(header='true', inferschema='false', nullValue='\\N')
events_df = events_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/events.csv") 

In [197]:
events_df.head(10)

[Row(display_id=1, uuid=u'cb8c55702adb93', document_id_e=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=2, uuid=u'79a85fa78311b9', document_id_e=1794259, timestamp=81, platform=2, geo_location=u'US>CA>807'),
 Row(display_id=3, uuid=u'822932ce3d8757', document_id_e=1179111, timestamp=182, platform=2, geo_location=u'US>MI>505'),
 Row(display_id=4, uuid=u'85281d0a49f7ac', document_id_e=1777797, timestamp=234, platform=2, geo_location=u'US>WV>564'),
 Row(display_id=5, uuid=u'8d0daef4bf5b56', document_id_e=252458, timestamp=338, platform=2, geo_location=u'SG>00'),
 Row(display_id=6, uuid=u'7765b4faae4ad4', document_id_e=1773517, timestamp=395, platform=3, geo_location=u'US>OH>510'),
 Row(display_id=7, uuid=u'2cc3f6457d16da', document_id_e=1149661, timestamp=602, platform=2, geo_location=u'US>MT>762'),
 Row(display_id=8, uuid=u'166fc654d73c98', document_id_e=1330329, timestamp=638, platform=2, geo_location=u'US>PA>566'),
 Row(display_id=9, uuid=u'9dddccf70f6067

In [132]:
document_entities_schema = StructType(
            [StructField("document_id", IntegerType(), True),
             StructField("entity_id", StringType(), True),
             StructField("confidence_level_ent", DoubleType(), True)]
             )

In [133]:
document_entities_df = spark.read.schema(document_entities_schema).options(header='true', inferschema='false', nullValue='\\N')
document_entities_df = document_entities_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/documents_entities.csv")

In [134]:
document_entities_df.head()

Row(document_id=1524246, entity_id=u'f9eec25663db4cd83183f5c805186f16', confidence_level_ent=0.672865314504701)

In [135]:
document_topics_schema = StructType(
            [StructField("document_id", IntegerType(), True),
             StructField("topic_id", IntegerType(), True),
             StructField("confidence_level_top", DoubleType(), True)]
             )

In [136]:
document_topics_df = spark.read.schema(document_topics_schema).options(header='true', inferschema='false', nullValue='\\N')
document_topics_df = document_topics_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/documents_topics.csv")

In [137]:
document_topics_df.head(10)

[Row(document_id=1595802, topic_id=140, confidence_level_top=0.0731131601068925),
 Row(document_id=1595802, topic_id=16, confidence_level_top=0.0594164867373976),
 Row(document_id=1595802, topic_id=143, confidence_level_top=0.0454207537554526),
 Row(document_id=1595802, topic_id=170, confidence_level_top=0.0388674285182961),
 Row(document_id=1524246, topic_id=113, confidence_level_top=0.196450402209685),
 Row(document_id=1524246, topic_id=260, confidence_level_top=0.14287816210395),
 Row(document_id=1524246, topic_id=92, confidence_level_top=0.0331591277639303),
 Row(document_id=1524246, topic_id=168, confidence_level_top=0.0140903418233461),
 Row(document_id=1524246, topic_id=54, confidence_level_top=0.00878222028049753),
 Row(document_id=1524246, topic_id=207, confidence_level_top=0.00828237207366929)]

In [138]:
document_entities_df.head(10)

[Row(document_id=1524246, entity_id=u'f9eec25663db4cd83183f5c805186f16', confidence_level_ent=0.672865314504701),
 Row(document_id=1524246, entity_id=u'55ebcfbdaff1d6f60b3907151f38527a', confidence_level_ent=0.399113728441297),
 Row(document_id=1524246, entity_id=u'839907a972930b17b125eb0247898412', confidence_level_ent=0.392095749652966),
 Row(document_id=1524246, entity_id=u'04d8f9a1ad48f126d5806a9236872604', confidence_level_ent=0.213996376305138),
 Row(document_id=1617787, entity_id=u'612a1d17685a498aff4f036c1ee02c16', confidence_level_ent=0.386192829940441),
 Row(document_id=1617787, entity_id=u'fb8c6cb0879e0de876298177eb1d3fcc', confidence_level_ent=0.364116465983032),
 Row(document_id=1617787, entity_id=u'793c6a6cf386edb820600d49df045475', confidence_level_ent=0.349168245516694),
 Row(document_id=1617787, entity_id=u'b525b84d5ed52a34565b8fb870555abe', confidence_level_ent=0.287004519610819),
 Row(document_id=1617787, entity_id=u'758cb9cb3014607cb4a691cbd485cc94', confidence_leve

In [139]:
document_meta_df.head(10)

[Row(document_id=1595802, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 5, 0, 0)),
 Row(document_id=1524246, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 5, 26, 11, 0)),
 Row(document_id=1617787, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 5, 27, 0, 0)),
 Row(document_id=1615583, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 7, 0, 0)),
 Row(document_id=1615460, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 20, 0, 0)),
 Row(document_id=1615354, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 10, 0, 0)),
 Row(document_id=1614611, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 5, 13, 0)),
 Row(document_id=1614235, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 9, 0, 0)),
 Row(document_id=1614225, source_id=1, publisher_id=603, publish_time=datetime.datetime(2016, 6, 9, 0, 0)),
 Row(document_id=14882

In [140]:
document_categories_df.head(10)

[Row(document_id=1595802, category_id=1611, confidence_level_cat=0.92),
 Row(document_id=1595802, category_id=1610, confidence_level_cat=0.07),
 Row(document_id=1524246, category_id=1807, confidence_level_cat=0.92),
 Row(document_id=1524246, category_id=1608, confidence_level_cat=0.07),
 Row(document_id=1617787, category_id=1807, confidence_level_cat=0.92),
 Row(document_id=1617787, category_id=1608, confidence_level_cat=0.07),
 Row(document_id=1615583, category_id=1305, confidence_level_cat=0.92),
 Row(document_id=1615583, category_id=1806, confidence_level_cat=0.07),
 Row(document_id=1615460, category_id=1613, confidence_level_cat=0.540646372),
 Row(document_id=1615460, category_id=1603, confidence_level_cat=0.041136137)]

In [141]:
page_views_sample_df = spark.read.schema(page_views_schema).options(header='true', inferschema='false', nullValue='\\N')
page_views_sample_df = page_views_sample_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/page_views_sample.csv") 

In [142]:
page_views_sample_df.head()

Row(uuid=u'1fd5f051fba643', document_id=120, timestamp=31905835, platform=1, geo_location=u'RS', traffic_source=2)

In [143]:
sample_submission_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", StringType(), True)]
                    )

In [144]:
sample_submission_df = spark.read.schema(sample_submission_schema).options(header='true', inferschema='false', nullValue='\\N')
sample_submission_df = sample_submission_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/sample_submission.csv") 

In [145]:
sample_submission_df.head(10)

[Row(display_id=16874594, ad_id=u'66758 150083 162754 170392 172888 180797'),
 Row(display_id=16874595, ad_id=u'8846 30609 143982'),
 Row(display_id=16874596, ad_id=u'11430 57197 132820 153260 173005 288385 289122 289915'),
 Row(display_id=16874597, ad_id=u'137858 143981 155945 180965 182039 285834 305790 308836'),
 Row(display_id=16874598, ad_id=u'67292 145937 250082 335632'),
 Row(display_id=16874599, ad_id=u'91681 163776 173130 210516 213116 296295'),
 Row(display_id=16874600, ad_id=u'2150 30682 57591 70529 114836 133050'),
 Row(display_id=16874601, ad_id=u'14082 92003 118470 129490 140942 190713'),
 Row(display_id=16874602, ad_id=u'131316 154918 268548 269017 281563'),
 Row(display_id=16874603, ad_id=u'44460 140423 156050 163139')]

In [146]:
promoted_content_schema = StructType(
                    [StructField("ad_id", IntegerType(), True),
                    StructField("document_id", IntegerType(), True),
                    StructField("campaign_id", IntegerType(), True),
                    StructField("advertiser_id", IntegerType(), True)]
                    )

In [147]:
promoted_content_df = spark.read.schema(promoted_content_schema).options(header='true', inferschema='false', nullValue='\\N')
promoted_content_df = promoted_content_df.csv("gs://dataproc-b10d4d38-fd55-4d88-8a9f-ba1154a55132-us/data/promoted_content.csv")

In [148]:
promoted_content_df.head(10)

[Row(ad_id=1, document_id=6614, campaign_id=1, advertiser_id=7),
 Row(ad_id=2, document_id=471467, campaign_id=2, advertiser_id=7),
 Row(ad_id=3, document_id=7692, campaign_id=3, advertiser_id=7),
 Row(ad_id=4, document_id=471471, campaign_id=2, advertiser_id=7),
 Row(ad_id=5, document_id=471472, campaign_id=2, advertiser_id=7),
 Row(ad_id=6, document_id=12736, campaign_id=1, advertiser_id=7),
 Row(ad_id=7, document_id=12808, campaign_id=1, advertiser_id=7),
 Row(ad_id=8, document_id=471477, campaign_id=2, advertiser_id=7),
 Row(ad_id=9, document_id=13379, campaign_id=1, advertiser_id=7),
 Row(ad_id=10, document_id=13885, campaign_id=1, advertiser_id=7)]

In [149]:
promoted_content_df.select('ad_id').distinct().count() == promoted_content_df.count()
#In case you want the list of values for that column, do promoted_content_df.select('ad_id').distinct().rdd.map(lambda r: r[0]).collect()

True

## Data Merging

First, the clicks and the events data set are merged using a left join. After that, I successively merged the various files related to documents to create one big joined table. In the end, the tables related to clicks, events and documents are merged together for the final table which is to be used to build a model on.

In [150]:
clicks_train_events_df = clicks_train_df.join(events_df, on=['display_id'], how='left')

In [151]:
clicks_train_events_df.count() == clicks_train_df.count()

True

In [152]:
clicks_test_events_df = clicks_test_df.join(events_df, on=['display_id'], how='left')

In [153]:
clicks_train_events_df.head(10)

[Row(display_id=148, ad_id=89351, clicked=1, uuid=u'9adce6a5363308', document_id=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=148, ad_id=152140, clicked=0, uuid=u'9adce6a5363308', document_id=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=148, ad_id=152656, clicked=0, uuid=u'9adce6a5363308', document_id=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=148, ad_id=326768, clicked=0, uuid=u'9adce6a5363308', document_id=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=463, ad_id=28270, clicked=0, uuid=u'85d999a1dd5483', document_id=1787133, timestamp=34492, platform=1, geo_location=u'US>SD>764'),
 Row(display_id=463, ad_id=38813, clicked=1, uuid=u'85d999a1dd5483', document_id=1787133, timestamp=34492, platform=1, geo_location=u'US>SD>764'),
 Row(display_id=463, ad_id=104911, clicked=0, uuid=u'85d999a1dd5483', document_id=1787133, timestamp=34492, platfor

The below code is just to validate if the join worked fine. 

In [154]:
clicks_train_df[clicks_train_df['display_id'] == 1].head(10)

[Row(display_id=1, ad_id=42337, clicked=0),
 Row(display_id=1, ad_id=139684, clicked=0),
 Row(display_id=1, ad_id=144739, clicked=1),
 Row(display_id=1, ad_id=156824, clicked=0),
 Row(display_id=1, ad_id=279295, clicked=0),
 Row(display_id=1, ad_id=296965, clicked=0)]

In [155]:
events_df[events_df['display_id'] == 148].head(10)

[Row(display_id=148, uuid=u'9adce6a5363308', document_id=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612')]

In [156]:
clicks_train_events_df[clicks_train_events_df['display_id'] == 1].head(10)

[Row(display_id=1, ad_id=42337, clicked=0, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=1, ad_id=139684, clicked=0, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=1, ad_id=144739, clicked=1, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=1, ad_id=156824, clicked=0, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=1, ad_id=279295, clicked=0, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519'),
 Row(display_id=1, ad_id=296965, clicked=0, uuid=u'cb8c55702adb93', document_id=379743, timestamp=61, platform=3, geo_location=u'US>SC>519')]

Now the above data frame is merged with the promoted_content data frame on the 'ad_id' field.

In [157]:
clicks_train_e_promo_cont_df = clicks_train_events_df.join(promoted_content_df, on=['ad_id'], how='left')

In [158]:
clicks_test_e_promo_cont_df = clicks_test_events_df.join(promoted_content_df, on=['ad_id'], how='left')

In [159]:
document_meta_df.select('document_id').distinct().count() == document_meta_df.count()

True

In [160]:
clicks_train_e_p_d_meta_df = clicks_train_e_promo_cont_df.join(document_meta_df, on=['document_id'], how='left')

AnalysisException: u"Reference 'document_id' is ambiguous, could be: document_id#826, document_id#908.;"

In [ ]:
clicks_train_e_promo_cont_df.head()

In [ ]:
events_df.select('display_id').distinct().count() == events_df.count()

In [ ]:
clicks_train_df.select('display_id').distinct().count() == clicks_train_df.count()

Since merging events and document info is giving an error, I'll first try to merge clicks and document information for finding out the information about what document and ad combinations lead to clicks.

In [173]:
c_train_pr_cont_df = clicks_train_df.join(promoted_content_df, on=['ad_id'], how='left')
c_test_pr_cont_df = clicks_test_df.join(promoted_content_df, on=['ad_id'], how='left')

In [174]:
clicks_train_df.select('ad_id').distinct().count() == clicks_train_df.count()

False

In [ ]:
This shows that an ad(ad_id) could be in multiple sets of recommendations.

In [172]:
c_train_pr_cont_df.head(10)

[Row(ad_id=148, display_id=9944564, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=3195987, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=3262691, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=3267986, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=3678937, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=3783605, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=4349019, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=8869255, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=148, display_id=8944408, clicked=0, document_id=471520, campaign_id=34, advertiser_id=19),
 Row(ad_id=471, display_id=9643855, clicked=0, document_id=76819, campaign_id=59, 

In [162]:
document_m_t_df = document_meta_df.join(document_topics_df, on=['document_id'], how='left')

In [167]:
document_m_t_df[document_m_t_df['document_id'] == 148].head(30)

[Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=153, confidence_level_top=0.0752369712571181),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=140, confidence_level_top=0.0719832117961455),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=8, confidence_level_top=0.0610842713790318),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=172, confidence_level_top=0.0420603297928532),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=244, confidence_level_top=0.0397126495989423),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=179, confidence_level_top=0.0368485664530311),
 Row(document_id=148, source_i

In [164]:
document_m_t_e_df = document_m_t_df.join(document_entities_df, on=['document_id'], how='left')

In [168]:
document_m_t_e_df[document_m_t_e_df['document_id'] == 148].head(30)

[Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=153, confidence_level_top=0.0752369712571181, entity_id=u'e1c74838563ef5d205063b0d95afa414', confidence_level_ent=0.632025758366959),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=153, confidence_level_top=0.0752369712571181, entity_id=u'6fd68f102042c6554cb2592fae942264', confidence_level_ent=0.404909109697693),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=153, confidence_level_top=0.0752369712571181, entity_id=u'ae3de5466bfa10459eebcbe02ac9b3ee', confidence_level_ent=0.276364701967659),
 Row(document_id=148, source_id=1787, publisher_id=118, publish_time=datetime.datetime(2008, 7, 1, 0, 0), topic_id=153, confidence_level_top=0.0752369712571181, entity_id=u'9da9595caa381755c9353ae7179f2117', confidence_level_ent=0.272666585584267),
 Row

In [169]:
document_m_t_e_df.count()

36728674

In [170]:
document_m_t_e_c_df = document_m_t_e_df.join(document_categories_df, on=['document_id'], how='left')

In [171]:
document_m_t_e_c_df.count()

72558106

In [176]:
c_train_pr_doc_det_df = c_train_pr_cont_df.join(document_m_t_e_c_df, on=['document_id'], how='left')
c_test_pr_doc_det_df = c_test_pr_cont_df.join(document_m_t_e_c_df, on=['document_id'], how='left')

In [177]:
c_train_pr_doc_det_df[c_train_pr_doc_det_df['document_id'] == 148].head(30)

[]

In [180]:
c_train_pr_doc_det_df.count()

1482391843

In [182]:
c_train_pr_doc_det_df.head(10)

[Row(document_id=29834, ad_id=45, display_id=15862091, clicked=0, campaign_id=26, advertiser_id=39, source_id=2586, publisher_id=278, publish_time=datetime.datetime(2016, 6, 28, 0, 0), topic_id=256, confidence_level_top=0.212099170435346, entity_id=None, confidence_level_ent=None, category_id=1513, confidence_level_cat=0.7712613),
 Row(document_id=29834, ad_id=45, display_id=15862091, clicked=0, campaign_id=26, advertiser_id=39, source_id=2586, publisher_id=278, publish_time=datetime.datetime(2016, 6, 28, 0, 0), topic_id=256, confidence_level_top=0.212099170435346, entity_id=None, confidence_level_ent=None, category_id=1514, confidence_level_cat=0.058682925),
 Row(document_id=29834, ad_id=45, display_id=15864222, clicked=0, campaign_id=26, advertiser_id=39, source_id=2586, publisher_id=278, publish_time=datetime.datetime(2016, 6, 28, 0, 0), topic_id=256, confidence_level_top=0.212099170435346, entity_id=None, confidence_level_ent=None, category_id=1513, confidence_level_cat=0.7712613),

In [246]:
clicks_train_df.count()

87141731

In [198]:
# c_train_pr_doc_det_ev_df = c_train_pr_doc_det_df.join(events_df, on=['document_id','display_id'], how='left')
# c_test_pr_doc_det_ev_df = c_test_pr_doc_det_df.join(events_df, on=['document_id','display_id'], how='left')
# ( c_train_pr_doc_det_df.document_id == events_df.document_id) & (c_train_pr_doc_det_df.display_id == events_df.display_id)
# c_train_pr_doc_det_ev_df = c_train_pr_doc_det_df.join(events_df, ( c_train_pr_doc_det_df.document_id == events_df.document_id) & (c_train_pr_doc_det_df.display_id == events_df.display_id), how='left')
c_train_pr_doc_det_ev_df = c_train_pr_doc_det_df.join(events_df, on=['display_id'], how='left')

In [185]:
c_train_pr_doc_det_ev_df.count()

1482391843

In [199]:
c_train_pr_doc_det_ev_df.head(30)

[Row(display_id=148, document_id=990613, ad_id=89351, clicked=1, campaign_id=7617, advertiser_id=2181, source_id=9457, publisher_id=None, publish_time=datetime.datetime(2015, 12, 9, 0, 0), topic_id=148, confidence_level_top=0.071109160240813, entity_id=u'956ae28fab890a14b653f114817152fb', confidence_level_ent=0.200192220085654, category_id=1408, confidence_level_cat=0.435300024, uuid=u'9adce6a5363308', document_id_e=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=148, document_id=990613, ad_id=89351, clicked=1, campaign_id=7617, advertiser_id=2181, source_id=9457, publisher_id=None, publish_time=datetime.datetime(2015, 12, 9, 0, 0), topic_id=148, confidence_level_top=0.071109160240813, entity_id=u'956ae28fab890a14b653f114817152fb', confidence_level_ent=0.200192220085654, category_id=1802, confidence_level_cat=0.033120654, uuid=u'9adce6a5363308', document_id_e=1205772, timestamp=11202, platform=2, geo_location=u'US>LA>612'),
 Row(display_id=148, documen

In [200]:
c_train_pr_doc_det_ev_df[c_train_pr_doc_det_ev_df['uuid'] != 'None'].count()

1482391843

In [245]:
events_df[events_df['display_id'] == '16004523'].head(10)

[Row(display_id=16004523, uuid=u'48136136ea989c', document_id_e=1714149, timestamp=1070958883, platform=1, geo_location=u'US>NY>501')]

In [211]:
est = GradientBoostingClassifier(n_estimators=200, max_depth=3)

In [216]:
c_train_pr_doc_det_ev_pandas_df_train_X = c_train_pr_doc_det_ev_df.select('display_id','document_id','ad_id','campaign_id','advertiser_id','source_id',
                                                                        'publisher_id','publish_time','topic_id','confidence_level_top','entity_id',
                                                                        'confidence_level_ent','category_id','confidence_level_cat','uuid',
                                                                        'document_id_e','timestamp','platform','geo_location')

In [217]:
c_train_pr_doc_det_ev_pandas_df_train_y = c_train_pr_doc_det_ev_df.select('clicked')

In [219]:
rf = RandomForestClassifier(numTrees=100, maxDepth=10, labelCol="indexed", seed=42)

In [231]:
c_train_pr_doc_det_ev_df_final = c_train_pr_doc_det_ev_df.select('display_id','document_id','ad_id','campaign_id','advertiser_id','source_id',
                                                                        'publisher_id','topic_id','confidence_level_top',
                                                                        'confidence_level_ent','category_id','confidence_level_cat',
                                                                        'document_id_e','platform','clicked')

In [ ]:
c_test_pr_doc_det_ev_df_final = c_test_pr_doc_det_ev_df.select('display_id','document_id','ad_id','campaign_id','advertiser_id','source_id',
                                                                        'publisher_id','topic_id','confidence_level_top',
                                                                        'confidence_level_ent','category_id','confidence_level_cat',
                                                                        'document_id_e','platform','clicked')

## Model building

Here is the part where all my attempst to build a model have failed so far. I've tried so many things that I don't even know what part relates to what attempt :P. 
To run a model in PySpark, a pipeline needs to be built. A simple one is attempted here.

In [247]:
cols_now = ['display_id','document_id','ad_id','campaign_id','advertiser_id','source_id',
'publisher_id','topic_id','confidence_level_top',
'confidence_level_ent','category_id','confidence_level_cat',
'document_id_e','platform']
assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
labelIndexer = StringIndexer(inputCol='clicked', outputCol="label")
tmp = [assembler_features, labelIndexer]
pipeline = Pipeline(stages=tmp)

In [242]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=cols_now,
    outputCol="features")

transformed = assembler.transform(c_train_pr_doc_det_ev_df_final)

from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

(transformed.select(col('clicked').alias("label"), col("features"))
  .rdd
  .map(lambda row: LabeledPoint(row.label, row.features)))

PythonRDD[1611] at RDD at PythonRDD.scala:48

In [243]:
transformed.head()

Row(display_id=148, document_id=1060089, ad_id=152140, campaign_id=19032, advertiser_id=1593, source_id=7744, publisher_id=1204, topic_id=227, confidence_level_top=0.334231753378017, confidence_level_ent=0.203818477105146, category_id=1403, confidence_level_cat=0.92, document_id_e=1205772, platform=2, clicked=0, features=DenseVector([148.0, 1060089.0, 152140.0, 19032.0, 1593.0, 7744.0, 1204.0, 227.0, 0.3342, 0.2038, 1403.0, 0.92, 1205772.0, 2.0]))

In [233]:
allData = pipeline.fit(c_train_pr_doc_det_ev_df_final).transform(c_train_pr_doc_det_ev_df_final)

In [234]:
allData.cache()

DataFrame[display_id: int, document_id: int, ad_id: int, campaign_id: int, advertiser_id: int, source_id: int, publisher_id: int, topic_id: int, confidence_level_top: double, confidence_level_ent: double, category_id: int, confidence_level_cat: double, document_id_e: int, platform: int, clicked: int, features: vector, label: double]

In [255]:
# Attempting a new pipeline
labelIndexer = StringIndexer(inputCol="clicked", outputCol="indexedLabel").fit(c_train_pr_doc_det_ev_df_final)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
#featureIndexer = VectorIndexer(inputCol=cols_now, outputCol='features'), outputCol="indexedFeatures", maxCategories=4).fit(c_train_pr_doc_det_ev_df_final)
featureIndexer = VectorIndexer(inputCol=transformed, outputCol="indexedFeatures", maxCategories=4).fit(c_train_pr_doc_det_ev_df_final)


TypeError: Invalid param value given for param "inputCol". Could not convert <class 'pyspark.sql.dataframe.DataFrame'> to string type

In [240]:


# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Train model.  This also runs the indexers.
model = pipeline.fit(c_train_pr_doc_det_ev_df)

# Make predictions.
# predictions = model.transform(testData)

NameError: name 'featureIndexer' is not defined

In [239]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(c_train_pr_doc_det_ev_df_final)

IllegalArgumentException: u'Field "features" does not exist.'

In [ ]:
# Make predictions.
predictions = model.transform(c_test_pr_doc_det_ev_df_final)

In [235]:
rf = RF(labelCol='label', featuresCol='features',numTrees=200)
fit = rf.fit(allData)

Py4JJavaError: An error occurred while calling o1142.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 388.0 failed 4 times, most recent failure: Lost task 0.3 in stage 388.0 (TID 12339, cluster-1-w-3.c.ultra-dimension-155718.internal): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<col1:double,col2:double,col3:double,col4:double,col5:double,col6:double,col7:double,col8:double,col9:double,col10:double,col11:double,col12:double,col13:double,col14:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:370)
	at org.apache.spark.sql.execution.columnar.InMemoryRelation$$anonfun$1$$anon$1.hasNext(InMemoryRelation.scala:139)
	at org.apac

In [ ]:
c_test_pr_doc_det_ev_df_final = c_test_pr_doc_det_ev_df.select('display_id','document_id','ad_id','campaign_id','advertiser_id','source_id',
                                                                        'publisher_id','topic_id','confidence_level_top',
                                                                        'confidence_level_ent','category_id','confidence_level_cat',
                                                                        'document_id_e','platform')

In [ ]:
test_data = pipeline.fit(c_test_pr_doc_det_ev_df_final).transform(c_test_pr_doc_det_ev_df_final)